In [88]:

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [89]:
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [90]:
import pandas as pd
import numpy as np

# [2, 5, 7, 8, 9, 13, 14] - [9, 14, 2, 7, 13, ]

# data 5 is baddest and 9 is largest

df = pd.read_csv("odds_datas/all_data.csv") # odds_data
df.rename(columns={"HT.1": 'AT'}, inplace=True)
df


,week,HT,AT,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,H_score,A_score
0,1,TOT,LUT,1.46,4.36,7.20,1.23,4.13,1.78,2.04,3.09,1.37,6.73,1.10,1,0
1,1,WHU,ASV,2.43,3.23,3.02,1.28,3.67,1.93,1.87,3.43,1.31,7.95,1.08,1,1
2,1,MNC,BRI,1.57,4.03,6.02,1.20,4.56,1.69,2.17,2.84,1.43,6.09,1.12,3,1
3,1,EVE,FUL,2.29,3.02,3.53,1.33,3.35,2.07,1.75,3.76,1.27,9.27,1.06,1,0
4,1,CRY,BOU,1.96,3.25,4.27,1.33,3.29,2.09,1.74,3.85,1.26,9.31,1.06,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16715,38,LUT,BOU,2.43,3.24,3.01,1.33,3.31,2.07,1.76,3.85,1.26,9.22,1.06,2,1
16716,38,MNC,NWC,1.77,3.63,4.72,1.20,4.47,1.71,2.14,2.86,1.42,6.20,1.12,0,1
16717,38,WOL,SHU,1.76,3.44,5.15,1.33,3.29,2.08,1.75,3.88,1.26,9.29,1.06,1,1
16718,38,ARS,LIV,2.12,3.18,3.75,1.24,4.01,1.81,2.00,3.04,1.38,6.81,1.10,0,0


In [91]:
# path = "odds_datas"
# records = [
#     f"{path}/test_1_odds_data.csv", f"{path}/test_2_odds_data.csv", f"{path}/test_3_odds_data.csv"
# ]

## Feature Engineering

In [92]:

class Team:
    def __init__(self, name):
        self.name = name
        self.goals_for = 0
        self.goals_against = 0
        self.goal_difference = 0
        self.points = 0
        self.position = 0
        self.last_results = []  # List to store last results
        self.last_1_result = 0
        self.last_2_result = 0
        
def update_table(team_1, team_1_score, team_2, team_2_score):
    team1 = next(team for team in team_objects if team.name == team_1)
    team2 = next(team for team in team_objects if team.name == team_2)

    scores = [team_1_score, team_2_score]
    new_teams = [team1, team2]

    for i, team in enumerate(new_teams):
        team.goals_for += scores[i]
        team.goals_against += scores[1 - i]
        team.goal_difference = team.goals_for - team.goals_against
        team.points += 3 if scores[i] > scores[1 - i] else 1 if scores[i] == scores[1 - i] else 0
        team.last_results.append(1 if scores[i] > scores[1 - i] else 0 if scores[i] == scores[1 - i] else -1)
        team.last_1_result = team.last_results[-1]
        team.last_2_result = team.last_results[-2] if len(team.last_results) > 1 else 0

    table = sorted(team_objects, key=lambda x: (-x.points, x.name.lower()))

    for i, team in enumerate(table):
        team.position = i + 1

    return table


def get_record(table_objects):
    table_dict = dict()
    for team in table_objects:
        table_dict[team.name] = (
            team.points,
            team.position,
            team.last_1_result,
            # team.last_2_result,
            # team.goals_for,
            # team.goals_against,
            team.goal_difference
        )

    return table_dict


def add_features(data):
    dff = data.copy()
    # print(dff.shape)

    col_list = []

    for i, row in dff.iterrows():
        ht, hcs, at, acs = row["HT"], row["H_score"], row["AT"], row["A_score"]

        table = update_table(ht, hcs, at, acs)

        if row["week"] > 2:
            val = table_dict[ht] + table_dict[at]
        else:
            val = (0,) * 8
        col_list.append(val)
        # print(val)

        if (i+1) % 20 == 0:
            table_dict = get_record(table)

    return col_list


# ------------------------------------------------------------------------------------------------


# Define function to check home and away scores and return result
def get_result(row):
    # if row['H_score'] > row['A_score']:
    #     return 1  # home_win = 1
    # elif row['H_score'] < row['A_score']:
    #     return 2  # away_win = 2
    # else:
    #     return 0  # draw = 0
    
    if row["H_score"] + row["A_score"] > 2:
        return 1
    else:
        return 0
        

def feature_engineering(data):
    df_result = data.copy()
        
    # Apply function to each row of the dataframe to create a new column
    df_result['result'] = df_result.apply(get_result, axis=1)

    df_result = df_result[df_result['week'] > 2]  # remove rows with incomplete values

    # Create a LabelEncoder object
    label_encoder = LabelEncoder()

    # Fit the LabelEncoder on the unique team names
    label_encoder.fit(teams)

    # Convert the categorical features to numeric using label encoding
    df_result['HT_encoded'] = label_encoder.transform(df_result['HT'])
    df_result['AT_encoded'] = label_encoder.transform(df_result['AT'])

    # remove irrelevant features
    df_result = df_result.drop(["H_score", "A_score", "HT", "AT"], axis=1)

    df_result = df_result.reset_index(drop=True)  # reset the index values

    # # Scaling the dataset using standardization method
    # X = df_result.drop("result", axis=1)
    # y = df_result["result"]

    # # assume that X is your dataset with numerical features
    # scaler = StandardScaler()
    # X_scaled = scaler.fit_transform(X)

    # # # concatenate the scaled numerical features with the Target variable feature
    # data_preprocessed = pd.concat([pd.DataFrame(X_scaled, columns=X.columns), y], axis=1)

    return df_result


In [93]:
df_encoded = df.copy()
# df_encoded = df_encoded.iloc[180: 5880]
# df_encoded = df_encoded.reset_index(drop=True)

df_encoded

,week,HT,AT,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,H_score,A_score
0,1,TOT,LUT,1.46,4.36,7.20,1.23,4.13,1.78,2.04,3.09,1.37,6.73,1.10,1,0
1,1,WHU,ASV,2.43,3.23,3.02,1.28,3.67,1.93,1.87,3.43,1.31,7.95,1.08,1,1
2,1,MNC,BRI,1.57,4.03,6.02,1.20,4.56,1.69,2.17,2.84,1.43,6.09,1.12,3,1
3,1,EVE,FUL,2.29,3.02,3.53,1.33,3.35,2.07,1.75,3.76,1.27,9.27,1.06,1,0
4,1,CRY,BOU,1.96,3.25,4.27,1.33,3.29,2.09,1.74,3.85,1.26,9.31,1.06,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16715,38,LUT,BOU,2.43,3.24,3.01,1.33,3.31,2.07,1.76,3.85,1.26,9.22,1.06,2,1
16716,38,MNC,NWC,1.77,3.63,4.72,1.20,4.47,1.71,2.14,2.86,1.42,6.20,1.12,0,1
16717,38,WOL,SHU,1.76,3.44,5.15,1.33,3.29,2.08,1.75,3.88,1.26,9.29,1.06,1,1
16718,38,ARS,LIV,2.12,3.18,3.75,1.24,4.01,1.81,2.00,3.04,1.38,6.81,1.10,0,0


In [94]:
# 7982

df_new = []

split_num = 380

total_seasons = len(df_encoded) // split_num
print("Total Seasons -", total_seasons)
start = 0
end = split_num
for v in range(total_seasons):
    # print(v+1)
    # print(start, end)
    df_temp = df_encoded.iloc[start: end].copy()

    teams = ['FOR', 'MNC', 'ASV', 'TOT', 'EVE', 'CHE', 'BRN', 'WHU', 'ARS', 'FUL', 
             'NWC', 'BOU', 'BUR', 'LIV', 'WOL', 'MNU', 'LUT', 'SHU', 'BRI', 'CRY']

    # teams = ['FOR', 'MNC', 'ASV', 'TOT', 'EVE', 'CHE', 'BRN', 'WHU', 'ARS', 'FUL',
    #          'NWC', 'BOU', 'LEI', 'LIV', 'WOL', 'MNU', 'LEE', 'SOU', 'BRI', 'CRY']
            
    # create a new empty league table to record scores & points
    team_objects = [Team(team) for team in teams]


    # adds scores & points from each league to the dataframe
    new_df = add_features(df_temp)
    # print(new_df)
    df_new += new_df  # add each dataframe to a list

    start += split_num
    end += split_num

cols = ["h_pts", "h_pos", "HL1R", "h_gd",
        "a_pts", "a_pos", "AL1R", "a_gd"]
records_df = pd.DataFrame(df_new, columns=cols)
records_df = pd.concat([df_encoded, records_df], axis=1)
# records_df

process_df = feature_engineering(records_df)

Total Seasons - 44


In [95]:
process_df

,week,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,h_pts,h_pos,HL1R,h_gd,a_pts,a_pos,AL1R,a_gd,result,HT_encoded,AT_encoded
0,3,8.62,4.90,1.36,1.21,4.45,1.67,2.20,2.85,1.42,5.92,1.13,1,15,-1,-1,2,13,0,0,1,5,0
1,3,1.46,4.38,7.18,1.21,4.35,1.73,2.11,2.95,1.40,6.30,1.12,1,17,0,-2,4,6,1,2,0,15,16
2,3,3.00,3.16,2.49,1.27,3.77,1.89,1.91,3.28,1.34,7.57,1.08,4,4,0,3,3,12,-1,-2,0,7,17
3,3,1.37,4.86,8.60,1.20,4.48,1.69,2.17,2.84,1.43,5.94,1.13,4,5,1,3,0,20,-1,-2,1,11,12
4,3,2.45,3.19,3.03,1.29,3.61,1.94,1.86,3.45,1.31,8.00,1.08,3,11,-1,-1,6,3,1,4,1,8,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15835,38,2.43,3.24,3.01,1.33,3.31,2.07,1.76,3.85,1.26,9.22,1.06,35,17,1,-20,39,14,-1,-13,1,12,2
15836,38,1.77,3.63,4.72,1.20,4.47,1.71,2.14,2.86,1.42,6.20,1.12,77,2,1,36,54,8,-1,3,0,13,15
15837,38,1.76,3.44,5.15,1.33,3.29,2.08,1.75,3.88,1.26,9.29,1.06,47,11,-1,-9,37,15,-1,-15,0,19,16
15838,38,2.12,3.18,3.75,1.24,4.01,1.81,2.00,3.04,1.38,6.81,1.10,72,3,1,31,77,1,1,25,0,0,11


In [96]:
process_df.isnull().sum()

week          0
home          0
draw          0
away          0
over_1        0
under_1       0
over_2        0
under_2       0
over_3        0
under_3       0
over_4        0
under_4       0
h_pts         0
h_pos         0
HL1R          0
h_gd          0
a_pts         0
a_pos         0
AL1R          0
a_gd          0
result        0
HT_encoded    0
AT_encoded    0
dtype: int64

In [97]:
total_season = len(df_encoded) / split_num
print("Total Seasons -", total_season)

Total Seasons - 44.0


In [98]:
duplicates = process_df[process_df.duplicated()].shape[0]

print("Total Duplicate Values:", duplicates)

# # Dropping duplicate values
# df_encoded.drop_duplicates(inplace=True)
# df_encoded.head()

Total Duplicate Values: 0


In [99]:

# over_number = 2
# lable = f"over_{over_number}_result"

# df["total"] = df["H_score"] + df["A_score"]
# df[lable] = np.where(df['total'] > over_number, 1, 0)
# # df = df.drop(["H_score", "A_score", "total"], axis=1)

# df_encoded = df.copy()
# df_encoded = df_encoded.drop(["HT", "AT", "H_score", "A_score", "total"], axis=1)

# # df_encoded = pd.get_dummies(df, columns=['HT', 'AT'])
# # df_encoded.replace({True: 1, False: 0}, inplace=True)

# df_encoded

# Model Building

In [100]:
lable = "result"

# Split the data into features and target variable
X = process_df.drop(lable, axis=1)
y = process_df[lable]

# Standardize the features using StandardScaler before splitting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create a dictionary to store the models and their names
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Support Vector Machine': SVC(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Gaussian Naive Bayes': GaussianNB(),
    'Multi-layer Perceptron': MLPClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42)
}

# Test each model and print the accuracy and F1 score
for model_name, model in models.items():
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate the accuracy and F1 score of the model
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Print the results
    print(f"Model: {model_name}")
    print('Accuracy:', accuracy)
    print('F1 score:', f1)
    print('-' * 30, "\n")

Model: Random Forest
Accuracy: 0.5186237373737373
F1 score: 0.518301426566092
------------------------------ 



c:\Users\CJay\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Logistic Regression
Accuracy: 0.5325126262626263
F1 score: 0.5231850038408328
------------------------------ 

Model: Support Vector Machine
Accuracy: 0.5328282828282829
F1 score: 0.5227534275703192
------------------------------ 

Model: K-Nearest Neighbors
Accuracy: 0.5205176767676768
F1 score: 0.5204192596674154
------------------------------ 

Model: Decision Tree
Accuracy: 0.48579545454545453
F1 score: 0.48587825845648386
------------------------------ 

Model: Gaussian Naive Bayes
Accuracy: 0.5318813131313131
F1 score: 0.5269830464569004
------------------------------ 



c:\Users\CJay\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model: Multi-layer Perceptron
Accuracy: 0.5148358585858586
F1 score: 0.5147087647992286
------------------------------ 

Model: Gradient Boosting
Accuracy: 0.5265151515151515
F1 score: 0.5187823703837887
------------------------------ 

Model: AdaBoost
Accuracy: 0.5258838383838383
F1 score: 0.5201999501351156
------------------------------ 

Model: XGBoost
Accuracy: 0.5202020202020202
F1 score: 0.5194959247791536
------------------------------ 



In [101]:

#          OVER 2
# Model: Support Vector Machine
# Accuracy: 0.5328282828282829
# F1 score: 0.5227534275703192
# ------------------------------ 

# Model: K-Nearest Neighbors
# Accuracy: 0.5205176767676768
# F1 score: 0.5204192596674154
# ------------------------------ 

# Model: Gaussian Naive Bayes
# Accuracy: 0.5318813131313131
# F1 score: 0.5269830464569004
# ------------------------------ 




#          OVER 1
# Model: Random Forest
# Accuracy: 0.7503156565656566
# F1 score: 0.6608930715065547
# ------------------------------ 

# Model: K-Nearest Neighbors
# Accuracy: 0.7130681818181818
# F1 score: 0.6673355588681249
# ------------------------------ 

In [109]:
lable = "result"

# Split the data into features and target variable
X = process_df.drop(lable, axis=1)
y = process_df[lable]

# Standardize the features using StandardScaler before splitting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create the KNeighborsClassifier model
knn = KNeighborsClassifier()

# Fit the model on the training data
knn.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn.predict(X_test)

# Calculate the accuracy and F1 score of the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the results
print("Model: KNeighborsClassifier")
print('Accuracy:', accuracy)
print('F1 score:', f1)
print('-' * 30)

Model: KNeighborsClassifier
Accuracy: 0.5205176767676768
F1 score: 0.5204192596674154
------------------------------


In [149]:
real = process_df["result"].head(100)

# print(list(real))
# list(pred_val)

win_count = 0
stake_count = 0
for a, b in enumerate(list(real)):
    val = process_df.drop("result", axis=1).iloc[a]
    # scaler = StandardScaler()
    # val = scaler.fit_transform(val)
    # print(val)
    pred_val = knn.predict(val.to_numpy().reshape(1, -1))

    if pred_val == 1:
        stake_count += 1
        if pred_val == b:
            print(pred_val, "-", b, "WIN")
            win_count += 1
        else:
            print(pred_val, "-", b, "Loss")

print("Total Play -", stake_count)
print("Total Won -", win_count, "\n")

[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 0 Loss
[1] - 0 Loss
[1] - 0 Loss
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 0 Loss
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 0 Loss
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
[1] - 0 Loss
[1] - 0 Loss
[1] - 1 WIN
[1] - 0 Loss
[1] - 1 WIN
Total Play - 62
Total Won - 36 



In [129]:
process_df.drop("result", axis=1).iloc[0:2]

,week,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,h_pts,h_pos,HL1R,h_gd,a_pts,a_pos,AL1R,a_gd,HT_encoded,AT_encoded
0,3,8.62,4.90,1.36,1.21,4.45,1.67,2.20,2.85,1.42,5.92,1.13,1,15,-1,-1,2,13,0,0,5,0
1,3,1.46,4.38,7.18,1.21,4.35,1.73,2.11,2.95,1.40,6.30,1.12,1,17,0,-2,4,6,1,2,15,16
